In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
   model_name = "unsloth/gemma-2-2b-it",
   max_seq_length = max_seq_length,
   dtype = dtype,
   load_in_4bit = load_in_4bit,)

/home/songchaewon/anaconda3/envs/sh/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.9: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.747 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [3]:
import pandas as pd
from datasets import Dataset

# CSV 파일을 pandas로 로드
df = pd.read_csv("./감정.csv")

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

In [5]:
import re

# 괄호 안의 단어 추출 함수
def net(input_string):
    # 각 항목이 리스트 형태일 경우 첫 번째 요소만 가져오기
    if isinstance(input_string, list):
        input_string = input_string[0]

    # 괄호 안의 단어를 추출하는 정규식 패턴
    pattern = r"\((.*?)\)"
    
    # 추출
    extracted_word = re.findall(pattern, input_string)
    return extracted_word

# 괄호 안의 단어와 괄호 자체를 제거하는 함수
def net2(input_string):
    # 각 항목이 리스트 형태일 경우 첫 번째 요소만 가져오기
    if isinstance(input_string, list):
        input_string = input_string[0]
    
    # 괄호 안의 단어와 괄호 자체를 제거하는 정규식 패턴
    pattern = r"\(.*?\)"
    
    # 제거
    cleaned_string = re.sub(pattern, "", input_string)
    return cleaned_string

# 프롬프트 생성 함수
def generate_prompt(example):
    output_texts = []
    for i in range(len(example)):
        emo = net(example['질문(감정)'][i])  # 각 항목에 대해 추출
        prompt = f"""다음의 문장에 대해 {emo}이 캐릭터에 어울리는 응답을 하시오.
문장:
{net2(example['질문(감정)'][i])}
응답:
{example['대답'][i]}<eos>
"""
        output_texts.append(prompt)
    return output_texts


train_data = dataset

# 프롬프트 생성
print(generate_prompt(train_data)[0])


다음의 문장에 대해 ['기쁨']이 캐릭터에 어울리는 응답을 하시오.
문장:
오늘 피곤해
응답:
피곤하구나! 그럴 땐 잠깐 스트레칭하고 기분 전환해보는 건 어때?<eos>



In [6]:
import pandas as pd
from datasets import Dataset

# CSV 파일을 pandas로 로드
df2 = pd.read_csv("./감정gpt.csv")

# Hugging Face Dataset으로 변환
dataset2 = Dataset.from_pandas(df2)

In [7]:
import re



# 프롬프트 생성 함수
def generate_prompt2(example):
    output_texts = []
    for i in range(len(example)):
         # 각 항목에 대해 추출
        prompt = f"""다음의 문장에서 느껴지는 감정을 '기쁨','슬픔','화남','짜증','불안' 이 다섯가지 감정 안에서 두개의 감정을 고르거나(한개의 감정보다는 되도록 2개의 감정을 골라줘) 해당하는 감정이 없으면 '좀더 자세히 말해주세요'라고 적어줘
                    ### 문장:
                    {example['문장'][i]}
                    ### 감정:
                    {example['감정'][i]}<eos>
                """
        output_texts.append(prompt)
    return output_texts


train_data2 = dataset2

# 프롬프트 생성
#print(generate_prompt2(train_data2)[0])


In [8]:
td={}
td2={}

td['text'] = generate_prompt(train_data)
td2['text'] = generate_prompt2(train_data2)

In [9]:
td2['text'][0]

"다음의 문장에서 느껴지는 감정을 '기쁨','슬픔','화남','짜증','불안' 이 다섯가지 감정 안에서 두개의 감정을 고르거나(한개의 감정보다는 되도록 2개의 감정을 골라줘) 해당하는 감정이 없으면 '좀더 자세히 말해주세요'라고 적어줘\n                    ### 문장:\n                    오늘 일이 너무 많아서 힘들어.\n                    ### 감정:\n                    불안, 슬픔<eos>\n                "

In [10]:
from datasets import load_dataset, concatenate_datasets
data={'text':[]}
EOS_TOKEN = tokenizer.eos_token
for i in range(len(td2['text'])):
    data['text'].append(td2['text'][i])
for i in range(len(td['text'])):
    data['text'].append(td['text'][i])
print(data)

{'text': ["다음의 문장에서 느껴지는 감정을 '기쁨','슬픔','화남','짜증','불안' 이 다섯가지 감정 안에서 두개의 감정을 고르거나(한개의 감정보다는 되도록 2개의 감정을 골라줘) 해당하는 감정이 없으면 '좀더 자세히 말해주세요'라고 적어줘\n                    ### 문장:\n                    오늘 일이 너무 많아서 힘들어.\n                    ### 감정:\n                    불안, 슬픔<eos>\n                ", "다음의 문장에서 느껴지는 감정을 '기쁨','슬픔','화남','짜증','불안' 이 다섯가지 감정 안에서 두개의 감정을 고르거나(한개의 감정보다는 되도록 2개의 감정을 골라줘) 해당하는 감정이 없으면 '좀더 자세히 말해주세요'라고 적어줘\n                    ### 문장:\n                    드디어 프로젝트가 끝났어!\n                    ### 감정:\n                    기쁨<eos>\n                ", "다음의 문장에서 느껴지는 감정을 '기쁨','슬픔','화남','짜증','불안' 이 다섯가지 감정 안에서 두개의 감정을 고르거나(한개의 감정보다는 되도록 2개의 감정을 골라줘) 해당하는 감정이 없으면 '좀더 자세히 말해주세요'라고 적어줘\n                    ### 문장:\n                    친구가 나한테 거짓말을 했어.\n                    ### 감정:\n                    화남, 슬픔<eos>\n                ", "다음의 문장에서 느껴지는 감정을 '기쁨','슬픔','화남','짜증','불안' 이 다섯가지 감정 안에서 두개의 감정을 고르거나(한개의 감정보다는 되도록 2개의 감정을 골라줘) 해당하는 감정이 없으면 '좀더 자세히 말해주세요'라고 적어줘\n                   

In [11]:
dataset = Dataset.from_dict(data)


In [12]:
dataset

Dataset({
    features: ['text'],
    num_rows: 200
})

In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field = "text",
    max_seq_length=512,
    packing = False,
    args=TrainingArguments(
        seed = 3407,
        output_dir = "최종",
        num_train_epochs = 25,
        #max_steps=3000,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        optim="adamw_8bit",
        warmup_steps=5,
        learning_rate=2e-4,
        lr_scheduler_type = "linear",
        fp16=not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps=1,
        weight_decay = 0.01,
    ),
 
    
)


Map: 100%|██████████| 200/200 [00:00<00:00, 18610.75 examples/s]


In [15]:
output_dir="최종"
# 파인튜닝 시작
import os

# 학습 체크포인트 폴더 유무를 체크합니다.
if not os.path.exists(output_dir):
    trainer_stats = trainer.train(resume_from_checkpoint = True)
else:
    trainer_stats = trainer.train() # 모델을 훈련시키고 통계를 반환

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 200 | Num Epochs = 25
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 625
 "-____-"     Number of trainable parameters = 20,766,720
  0%|          | 1/625 [00:35<6:11:14, 35.70s/it]

{'loss': 3.1309, 'grad_norm': 2.530071496963501, 'learning_rate': 4e-05, 'epoch': 0.04}


  0%|          | 2/625 [00:37<2:41:22, 15.54s/it]

{'loss': 3.1713, 'grad_norm': 2.5376625061035156, 'learning_rate': 8e-05, 'epoch': 0.08}


  0%|          | 3/625 [00:38<1:33:54,  9.06s/it]

...

{'loss': 0.0535, 'grad_norm': 0.1568756103515625, 'learning_rate': 3.2258064516129035e-07, 'epoch': 24.96}


100%|██████████| 625/625 [15:06<00:00,  1.41s/it]

{'loss': 0.0551, 'grad_norm': 0.15315957367420197, 'learning_rate': 0.0, 'epoch': 25.0}


100%|██████████| 625/625 [15:07<00:00,  1.45s/it]

{'train_runtime': 907.5114, 'train_samples_per_second': 5.51, 'train_steps_per_second': 0.689, 'train_loss': 0.13532728731632232, 'epoch': 25.0}


In [26]:
prompt = """다음의 문장에 대해 ['{}']이 캐릭터에 어울리는 응답을 하시오.
문장:
{}
응답:
"""
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        # "Continue the fibonacci sequence: 1, 1, 2, 3, 5, 8,", # instruction
        "기쁨", # instruction
        "남친이랑 헤어졌어.. 너무 보고싶다", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)



['<bos><bos><start_of_turn>user\n    다음의 문장에 대해 기쁨이 캐릭터에 어울리는 응답을 하시오.\n    문장: 남친이랑 헤어졌어.. 너무 보고싶다\n    <end_of_turn>\n    <start_of_turn>model--\n    그래도 힘내자! 금방 내가 옆에 있을게!<eos>']

In [17]:
prompt2 = """다음의 문장에서 느껴지는 감정을 '기쁨','슬픔','화남','짜증','불안' 이 다섯가지 감정 안에서 두개의 감정을 고르거나(한개의 감정보다는 되도록 2개의 감정을 골라줘) 해당하는 감정이 없으면 '좀더 자세히 말해주세요'라고 적어줘

### 문장:
{}

### 감정:
"""
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt2.format(
        # "Continue the fibonacci sequence: 1, 1, 2, 3, 5, 8,", # instruction
        "집가고 싶다..", # instruction
        # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 164, use_cache = True)
tokenizer.batch_decode(outputs)



["<bos>다음의 문장에서 느껴지는 감정을 '기쁨','슬픔','화남','짜증','불안' 이 다섯가지 감정 안에서 두개의 감정을 고르거나(한개의 감정보다는 되도록 2개의 감정을 골라줘) 해당하는 감정이 없으면 '좀더 자세히 말해주세요'라고 적어줘\n\n### 문장:\n집가고 싶다..\n\n### 감정:\n불안, 슬픔<eos>"]

In [ ]:
from huggingface_hub import create_repo

# Create the repository
create_repo("1chae/emotion_conversation", private=False, token="토큰")


In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 로컬에 PyTorch 포맷으로 모델 저장
model.save_pretrained("./test5")  # 모델을 로컬 디렉토리에 저장
tokenizer.save_pretrained("./test5")  # 토크나이저도 저장


('./test5/tokenizer_config.json',
 './test5/special_tokens_map.json',
 './test5/tokenizer.model',
 './test5/added_tokens.json',
 './test5/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 허브에 업로드
model.push_to_hub("1chae/emotion_conversation", use_auth_token="토큰")
tokenizer.push_to_hub("1chae/emotion_conversation", use_auth_token="토큰")
